# 02 — Exploratory Analysis (SQL-first)
**Formula 1 Race Strategy Analytics** — End-to-end data analytics project

**Goal:** Run core exploratory analyses directly from SQL (SQLite), then produce simple visuals to support a future Power BI dashboard.

In [ ]:

import os, sqlite3, pandas as pd, numpy as np
import matplotlib.pyplot as plt

CANDIDATE_PATHS = ["../data/f1.db", "/content/sample_f1_data/f1.db", "data/f1.db"]
DB_PATH = next((p for p in CANDIDATE_PATHS if os.path.exists(p)), None)
if DB_PATH is None:
    raise FileNotFoundError("Could not find f1.db. Upload it or set DB_PATH manually.")
print("Using database:", DB_PATH)

def q(sql: str) -> pd.DataFrame:
    with sqlite3.connect(DB_PATH) as conn:
        return pd.read_sql(sql, conn)


In [ ]:
q("SELECT name FROM sqlite_master WHERE type='table' ORDER BY 1;")

In [ ]:

wins_sql = '''
SELECT d.givenName || ' ' || d.familyName AS driver,
       COUNT(*) AS wins
FROM results r
JOIN drivers d ON d.driverId = r.driverId
WHERE r.position GLOB '[0-9]*' AND CAST(r.position AS INTEGER) = 1
GROUP BY r.driverId
ORDER BY wins DESC, driver ASC;
'''
wins = q(wins_sql); wins


In [ ]:

plt.figure(figsize=(6,4))
plt.bar(wins['driver'], wins['wins'])
plt.title("Wins per Driver")
plt.ylabel("Wins")
plt.xticks(rotation=20, ha='right'); plt.tight_layout(); plt.show()


In [ ]:

avg_points_sql = '''
SELECT d.givenName || ' ' || d.familyName AS driver,
       ROUND(AVG(CAST(r.points AS REAL)), 2) AS avg_points
FROM results r
JOIN drivers d ON d.driverId = r.driverId
WHERE r.points GLOB '[0-9]*'
GROUP BY r.driverId
ORDER BY avg_points DESC, driver ASC;
'''
avg_points = q(avg_points_sql); avg_points


In [ ]:

plt.figure(figsize=(6,4))
plt.bar(avg_points['driver'], avg_points['avg_points'])
plt.title("Average Points per Driver")
plt.ylabel("Avg Points")
plt.xticks(rotation=20, ha='right'); plt.tight_layout(); plt.show()


In [ ]:

positions_gained_sql = '''
WITH clean AS (
  SELECT CAST(grid AS INTEGER) AS grid,
         CAST(position AS INTEGER) AS finish,
         driverId
  FROM results
  WHERE grid GLOB '[0-9]*' AND position GLOB '[0-9]*'
)
SELECT d.givenName || ' ' || d.familyName AS driver,
       ROUND(AVG(grid - finish), 2) AS avg_positions_gained
FROM clean c
JOIN drivers d ON d.driverId = c.driverId
GROUP BY c.driverId
ORDER BY avg_positions_gained DESC, driver ASC;
'''
pos_gain = q(positions_gained_sql); pos_gain


In [ ]:

plt.figure(figsize=(6,4))
plt.bar(pos_gain['driver'], pos_gain['avg_positions_gained'])
plt.title("Average Positions Gained per Driver")
plt.ylabel("Positions Gained")
plt.axhline(0)
plt.xticks(rotation=20, ha='right'); plt.tight_layout(); plt.show()


In [ ]:

pit_team_sql = '''
WITH pit AS (
  SELECT season, round, driverId, CAST(duration AS REAL) AS dur
  FROM pit_stops
  WHERE duration != ''
)
SELECT c.name AS constructor,
       ROUND(AVG(p.dur), 2) AS avg_pit_s
FROM pit p
JOIN results r ON r.season = p.season AND r.round = p.round AND r.driverId = p.driverId
JOIN constructors c ON c.constructorId = r.constructorId
GROUP BY c.constructorId
ORDER BY avg_pit_s ASC;
'''
pit_team = q(pit_team_sql); pit_team


In [ ]:

plt.figure(figsize=(6,4))
plt.bar(pit_team['constructor'], pit_team['avg_pit_s'])
plt.title("Average Pit-stop Duration by Team (s)")
plt.ylabel("Seconds")
plt.xticks(rotation=20, ha='right'); plt.tight_layout(); plt.show()


In [ ]:

grid_finish_sql = '''
SELECT 
  CAST(r.grid     AS INTEGER)  AS grid,
  CAST(r.position AS INTEGER)  AS finish,
  r.status
FROM results r
WHERE r.grid GLOB '[0-9]*'
  AND r.position GLOB '[0-9]*'
  AND CAST(r.grid AS INTEGER) > 0
  AND CAST(r.position AS INTEGER) > 0;
'''
grid_finish = q(grid_finish_sql); grid_finish.head()


In [ ]:

pearson_r  = grid_finish[['grid','finish']].corr().loc['grid','finish']
spearman_r = grid_finish[['grid','finish']].corr(method='spearman').loc['grid','finish']
print("Pearson r:", round(float(pearson_r), 3))
print("Spearman r:", round(float(spearman_r), 3))


In [ ]:

x = grid_finish['grid'].astype(int).values
y = grid_finish['finish'].astype(int).values
m, b = np.polyfit(x, y, 1)

plt.figure(figsize=(5,5))
plt.scatter(x, y)
plt.plot(x, m*x + b)
plt.title("Qualifying Grid vs Finish (lower = better)")
plt.xlabel("Grid")
plt.ylabel("Finish")
plt.gca().invert_yaxis()
plt.tight_layout(); plt.show()


In [ ]:

circuit_perf_sql = '''
SELECT
  ci.circuitName AS circuit,
  ROUND(AVG(CAST(r.position AS REAL)), 2) AS avg_finish
FROM results r
JOIN races ra ON ra.season = r.season AND ra.round = r.round
JOIN circuits ci ON ci.circuitId = ra.circuitId
WHERE r.position GLOB '[0-9]*'
GROUP BY ci.circuitId
ORDER BY avg_finish ASC;
'''
circuit_perf = q(circuit_perf_sql); circuit_perf


In [ ]:

plt.figure(figsize=(6,4))
plt.bar(circuit_perf['circuit'], circuit_perf['avg_finish'])
plt.title("Average Finish by Circuit (lower = better)")
plt.ylabel("Avg Finish")
plt.xticks(rotation=20, ha='right'); plt.tight_layout(); plt.show()
